# 如何为检索器结果添加分数

[检索器](/docs/concepts/retrievers/)将返回[Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html)对象的序列，这些对象默认不包含有关检索过程的信息（例如，与查询的相似性分数）。在这里，我们演示如何将检索分数添加到文档的`.metadata`中：
1. 从[向量存储检索器](/docs/how_to/vectorstore_retriever)；
2. 从更高阶的LangChain检索器，例如[SelfQueryRetriever](/docs/how_to/self_query)或[MultiVectorRetriever](/docs/how_to/multi_vector)。

对于(1)，我们将在相应的[向量存储](/docs/concepts/vectorstores/)周围实现一个简短的包装函数。对于(2)，我们将更新相应类的方法。

## 创建向量存储

首先，我们用一些数据填充一个向量存储。我们将使用[PineconeVectorStore](https://python.langchain.com/api_reference/pinecone/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html)，但本指南兼容任何实现了`.similarity_search_with_score`方法的LangChain向量存储。

In [ ]:
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

docs = [
    Document(
        page_content="一群科学家带回了恐龙，混乱随之而来",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="莱昂纳多·迪卡普里奥迷失在一个梦中又一个梦中又一个梦中……",
        metadata={"year": 2010, "director": "克里斯托弗·诺兰", "rating": 8.2},
    ),
    Document(
        page_content="一位心理学家/侦探迷失在一系列梦中，而《盗梦空间》重复使用了这个想法",
        metadata={"year": 2006, "director": "今敏", "rating": 8.6},
    ),
    Document(
        page_content="一群普通大小的女性极其善良，一些男性对她们念念不忘",
        metadata={"year": 2019, "director": "格蕾塔·葛韦格", "rating": 8.3},
    ),
    Document(
        page_content="玩具活了过来，并乐在其中",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="三个人走进了禁区，三个人走出了禁区",
        metadata={
            "year": 1979,
            "director": "安德烈·塔可夫斯基",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

vectorstore = PineconeVectorStore.from_documents(
    docs, index_name="sample", embedding=OpenAIEmbeddings()
)

## 检索器

为了从向量存储检索器中获取分数，我们将底层向量存储的`.similarity_search_with_score`方法包装在一个简短的函数中，该函数将分数打包到相关文档的元数据中。

我们在函数中添加一个`@chain`装饰器，以创建一个[Runnable](/docs/concepts/lcel)，可以像典型的检索器一样使用。

In [3]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vectorstore.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [ ]:
result = retriever.invoke("恐龙")
result

(Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993.0, 'score': 0.84429127}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995.0, 'score': 0.792038262}),
 Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979.0, 'score': 0.751571238}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006.0, 'score': 0.747471571}))

请注意，检索步骤中的相似性分数已包含在上述文档的元数据中。

## SelfQueryRetriever

`SelfQueryRetriever`将使用LLM生成一个可能是结构化的查询——例如，它可以在通常的语义相似性驱动选择的基础上构建检索过滤器。有关更多详细信息，请参阅[本指南](/docs/how_to/self_query)。

`SelfQueryRetriever`包含一个简短的（1-2行）方法`_get_docs_with_query`，该方法执行`vectorstore`搜索。我们可以子类化`SelfQueryRetriever`并覆盖此方法以传播相似性分数。

首先，按照[操作指南](/docs/how_to/self_query)，我们需要建立一些元数据以进行过滤：

In [ ]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="电影的类型。可选值为['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="电影的上映年份",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="电影导演的名字",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="电影的评分（1-10）", type="float"
    ),
]
document_content_description = "电影的简要概述"
llm = ChatOpenAI(temperature=0)

然后我们覆盖`_get_docs_with_query`以使用底层向量存储的`similarity_search_with_score`方法：

In [ ]:
from typing import Any, Dict


class CustomSelfQueryRetriever(SelfQueryRetriever):
    def _get_docs_with_query(
        self, query: str, search_kwargs: Dict[str, Any]
    ) -> List[Document]:
        """获取文档，并添加分数信息。"""
        docs, scores = zip(
            *self.vectorstore.similarity_search_with_score(query, **search_kwargs)
        )
        for doc, score in zip(docs, scores):
            doc.metadata["score"] = score

        return docs

调用此检索器现在将在文档元数据中包含相似性分数。请注意，`SelfQueryRetriever`的底层结构化查询功能得以保留。

In [ ]:
retriever = CustomSelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)


result = retriever.invoke("评分低于8的恐龙电影")
result

(Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993.0, 'score': 0.84429127}),)

## MultiVectorRetriever

`MultiVectorRetriever`允许您将多个向量与单个文档关联。这在许多应用中可能很有用。例如，我们可以索引较大文档的小块并对这些块进行检索，但在调用检索器时返回较大的“父”文档。[ParentDocumentRetriever](/docs/how_to/parent_document_retriever/)，`MultiVectorRetriever`的子类，包含用于填充向量存储以支持此功能的便利方法。更多应用详见[操作指南](/docs/how_to/multi_vector/)。

为了通过此检索器传播分数，我们可以再次子类化`MultiVectorRetriever`并覆盖一个方法。这次我们将覆盖`_get_relevant_documents`。

首先，我们准备一些虚假的数据。我们生成虚假的“完整文档”并将其存储在文档存储中；这里我们将使用一个简单的[InMemoryStore](https://python.langchain.com/api_reference/core/stores/langchain_core.stores.InMemoryBaseStore.html)。

In [ ]:
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 父文档的存储层
docstore = InMemoryStore()
fake_whole_documents = [
    ("fake_id_1", Document(page_content="虚假的完整文档1")),
    ("fake_id_2", Document(page_content="虚假的完整文档2")),
]
docstore.mset(fake_whole_documents)

接下来，我们将一些虚假的“子文档”添加到我们的向量存储中。我们可以通过在其元数据中填充`"doc_id"`键将这些子文档链接到父文档。

In [ ]:
docs = [
    Document(
        page_content="来自较大文档的讨论猫的片段。",
        metadata={"doc_id": "fake_id_1"},
    ),
    Document(
        page_content="来自较大文档的讨论话语的片段。",
        metadata={"doc_id": "fake_id_1"},
    ),
    Document(
        page_content="来自较大文档的讨论巧克力的片段。",
        metadata={"doc_id": "fake_id_2"},
    ),
]

vectorstore.add_documents(docs)

['62a85353-41ff-4346-bff7-be6c8ec2ed89',
 '5d4a0e83-4cc5-40f1-bc73-ed9cbad0ee15',
 '8c1d9a56-120f-45e4-ba70-a19cd19a38f4']

为了传播分数，我们子类化`MultiVectorRetriever`并覆盖其`_get_relevant_documents`方法。在这里我们将进行两个更改：

1. 我们将使用底层向量存储的`similarity_search_with_score`方法将相似性分数添加到相应“子文档”的元数据中；
2. 我们将在检索到的父文档的元数据中包含这些子文档的列表。这将显示检索识别的文本片段及其对应的相似性分数。

In [ ]:
from collections import defaultdict

from langchain.retrievers import MultiVectorRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun


class CustomMultiVectorRetriever(MultiVectorRetriever):
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """获取与查询相关的文档。
        参数：
            query: 用于查找相关文档的字符串
            run_manager: 要使用的回调处理程序
        返回：
            相关文档列表
        """
        results = self.vectorstore.similarity_search_with_score(
            query, **self.search_kwargs
        )

        # 将doc_ids映射到子文档列表，并将分数添加到元数据
        id_to_doc = defaultdict(list)
        for doc, score in results:
            doc_id = doc.metadata.get("doc_id")
            if doc_id:
                doc.metadata["score"] = score
                id_to_doc[doc_id].append(doc)

        # 获取与doc_ids对应的文档，并在元数据中保留子文档
        docs = []
        for _id, sub_docs in id_to_doc.items():
            docstore_docs = self.docstore.mget([_id])
            if docstore_docs:
                if doc := docstore_docs[0]:
                    doc.metadata["sub_docs"] = sub_docs
                    docs.append(doc)

        return docs

调用此检索器，我们可以看到它识别了正确的父文档，包括具有相似性分数的子文档中的相关片段。

In [ ]:
retriever = CustomMultiVectorRetriever(vectorstore=vectorstore, docstore=docstore)

retriever.invoke("猫")

[Document(page_content='fake whole document 1', metadata={'sub_docs': [Document(page_content='A snippet from a larger document discussing cats.', metadata={'doc_id': 'fake_id_1', 'score': 0.831276655})]})]